In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from util import prepare_df

In [ ]:
df1 = prepare_df(xp_identifier="scheduling1")
df2 = prepare_df(xp_identifier="scheduling2")

In [ ]:
df1["xp"] = "wide"
df2["xp"] = "narrow"
df2["tid"] += df1["tid"].max() + 1
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
sns.set_theme(style="whitegrid", palette="deep")

In [ ]:
s = df["scheduler"] == df["scheduler"].iloc[0]
df_common = df.loc[s, ["tid", "EDFVD_test", "U", "xp"]].set_index("tid")

In [ ]:
df_res = df.pivot(index="tid", columns="scheduler", values="schedulable")
df_res = df_res.join(df_common)

In [ ]:
df_melt = df_res.melt(
    id_vars=["U", "xp"],
    value_vars=["EDF-VD", "LWLF", "EDFVD_test"],
    value_name="Schedulable",
)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(12, 5.5129), sharey=True)

ax_wide = axs[0]
ax_narrow = axs[1]

s = df_melt["xp"] == "wide"
sns.lineplot(
    data=df_melt.loc[s],
    x="U",
    y="Schedulable",
    hue="variable",
    errorbar=None,
    ax=ax_wide,
)

s = df_melt["xp"] == "narrow"
sns.lineplot(
    data=df_melt.loc[s],
    x="U",
    y="Schedulable",
    hue="variable",
    errorbar=None,
    ax=ax_narrow,
)

ax_wide.set_ylabel("Acceptance ratio")
ax_wide.set_xlabel("Average utilisation")
ax_narrow.set_xlabel("Average utilisation")

f.savefig(
    f"./{datetime.now().strftime('%Y%m%d_%H%M%S')}_scheduling.pdf", bbox_inches="tight"
)